In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
dfproducts = pd.read_csv(".\\data"+ "\\annex-6-1.csv", dtype = {"hs4": str, "hs4_o": str})

In [3]:
is_energy = dfproducts.high_catagory == "3. Energy"

energy_products = list(dfproducts.loc[is_energy,"hs4_o"])

energy_products.remove("27111990")

print(energy_products)

['2701', '2709', '271012250', '271111', '271112', '271113', '271311', '271312', '290511']


In [4]:
trade_type = "exports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

end_use = "hs?get=ALL_VAL_MO,UNIT_QY1,QTY_1_MO"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

In [5]:
def get_energy_quantity(commodity, url):

    url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=HS10" + "&E_COMMODITY=" + commodity + "*"

    url = url + "&CTY_CODE=5700"

    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    
    df["hs4_o"] = commodity
    
    #df["hs4"] = df["E_COMMODITY"].str[0:4]
    
    df["hs6"] = df["E_COMMODITY"].str[0:6]
    
    print(df["UNIT_QY1"].unique(), len(df["UNIT_QY1"].unique()))
    
    df["china_exports"] = df["ALL_VAL_MO"].astype(float)
    
    df["china_exports_quant"] = df["QTY_1_MO"].astype(float)
    
    df["quant_type"] = df["UNIT_QY1"].astype(str)
    
    df.drop(["ALL_VAL_MO", "E_COMMODITY", "COMM_LVL", "QTY_1_MO", "UNIT_QY1"], axis = 1, inplace = True)
    
    return df

In [6]:
df = get_energy_quantity("271012250", url)

<Response [200]>
['BBL'] 1


In [7]:
energy = pd.DataFrame([])

for xxx in energy_products:
    
    print(xxx)
    energy = energy.append(get_energy_quantity(xxx, url))
    

2701
<Response [200]>
['T'] 1
2709
<Response [200]>
['BBL'] 1
271012250
<Response [200]>
['BBL'] 1
271111
<Response [200]>
['M3'] 1
271112
<Response [200]>
['BBL' 'M3'] 2
271113
<Response [200]>
['BBL'] 1
271311
<Response [200]>
['T'] 1
271312
<Response [200]>
['T'] 1
290511
<Response [200]>
['L'] 1


In [8]:
energy.tail()

,time,CTY_CODE,hs4_o,hs6,china_exports,china_exports_quant,quant_type
87,2020-04-01,5700,290511,290511,47433.0,6962.0,L
88,2020-05-01,5700,290511,290511,199590.0,58572.0,L
89,2020-06-01,5700,290511,290511,247440.0,89182.0,L
90,2020-07-01,5700,290511,290511,337838.0,351566.0,L
91,2020-08-01,5700,290511,290511,307627.0,174207.0,L


In [9]:
df_energy = energy.merge(dfproducts.loc[is_energy,:], left_on = "hs4_o", right_on = "hs4_o", how = "left", indicator = True)

In [10]:
cnames = ['time', 'CTY_CODE', 'china_exports', 'hs6', 'china_exports_quant', 'quant_type',
          'hs4', 'hs4_o', 'description', 'low_catagory', 'high_catagory', '_merge']

In [11]:
df_energy.head()

,time,CTY_CODE,hs4_o,hs6,china_exports,china_exports_quant,quant_type,description,low_catagory,high_catagory,hs4,_merge
0,2013-01-01,5700,2701,270111,7837.0,11.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
1,2013-02-01,5700,2701,270111,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
2,2013-03-01,5700,2701,270111,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
3,2013-04-01,5700,2701,270111,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
4,2013-05-01,5700,2701,270111,7124.0,10.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both


In [12]:
df_energy[df_energy.low_catagory == "Coal"]

,time,CTY_CODE,hs4_o,hs6,china_exports,china_exports_quant,quant_type,description,low_catagory,high_catagory,hs4,_merge
0,2013-01-01,5700,2701,270111,7837.0,11.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
1,2013-02-01,5700,2701,270111,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
2,2013-03-01,5700,2701,270111,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
3,2013-04-01,5700,2701,270111,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
4,2013-05-01,5700,2701,270111,7124.0,10.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
...,...,...,...,...,...,...,...,...,...,...,...,...
334,2020-04-01,5700,2701,270120,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
335,2020-05-01,5700,2701,270120,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
336,2020-06-01,5700,2701,270120,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both
337,2020-07-01,5700,2701,270120,0.0,0.0,T,"Coal; briquett,es ovoids and similar solid fu...",Coal,3. Energy,2701,both


In [13]:
out_file = ".\\data"+ "\\phaseone-energy.parquet"

pq.write_table(pa.Table.from_pandas(df_energy), out_file)